In [1]:
from IPython.display import HTML

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=True;
    $('div.input').show()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Hide Code"></form>''')

In [ ]:
# Boilerplate code generated by OVITO Pro 3.6.0
from ovito.io import *
from ovito.modifiers import *
from ovito.data import *
from ovito.pipeline import *
from ovito.vis import *
import PySide2.QtCore
import IPython.display
import os
import pickle
import pb_sims as pb
from shutil import copyfile
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import math


In [5]:
def crop_image(img,margin=10):    
    x_img = img.shape[0]
    y_img = img.shape[1]
    x_mid = math.floor(img.shape[0]/2)
    y_mid = math.floor(img.shape[1]/2)

    # print(img[:,y_mid])
    v_strip = img[:,y_mid,0]

    v_strip_f = np.flip(v_strip)

    # find top
    r = 1
    i = 0
    while r == 1:
        r = v_strip[i]
        i += 1
    top = i

    # find bottom
    r = 1
    i = 0
    while r == 1:
        r = v_strip_f[i]
        i += 1
    bot = (x_img - i)

    mid_wall = top+50

    h_strip = img[mid_wall,:,0]
    h_strip_f = np.flip(h_strip)

    # find left
    r = 1
    i = 0
    while r == 1:
        r = h_strip[i]
        i += 1
    left = i
    # find right
    flip_mid_wall = np.flip(img[mid_wall,:],0)
    r = 1
    i = 0
    while r ==1:
        r = h_strip_f[i]
        i += 1

    right = y_img - i

    x_s = top - margin
    x_e = bot + margin
    y_s = left - margin
    y_e = right + margin
    x_c = [x_s,x_e]
    y_c = [y_s,y_e]
    cropped_image = img[x_c[0]:x_c[1],y_c[0]:y_c[1]]
    return cropped_image

In [6]:
def my_mod(pipeline,mol,cutoff,cal_ints,transp):
    # Manual modifications of the imported data objects:
    def modify_pipeline_input(frame: int, data: DataCollection):
        data.particles_.particle_types_.type_by_id_(1).radius = 0.5612
#         data.particles_.particle_types_.type_by_id_(2).color = (1.0, 0.4, 0.4)
        data.particles_.particle_types_.type_by_id_(2).radius = 0.5612
        data.particles_.particle_types_.type_by_id_(3).radius = 0.5612
        try:
            data.particles_.particle_types_.type_by_id_(4).radius = 0.5612
        except:
            pass
    pipeline.modifiers.append(modify_pipeline_input)

    # Visual element initialization:
    data = pipeline.compute() # Evaluate new pipeline to gain access to visual elements associated with the imported data objects.
    data.particles.vis.radius = 0.27
    data.cell.vis.enabled = False
    del data

#     del data # Done accessing input DataCollection of pipeline.
    pipeline.add_to_scene()

    # PBB-ECS-1 - Coordination analysis:
    pipeline.modifiers.append(CoordinationAnalysisModifier(
        cutoff = 4.6, 
        number_of_bins = 249, 
        partial = True))

    # PBB-ECS-1 - Expression selection:
    pipeline.modifiers.append(ExpressionSelectionModifier(expression = 'ParticleType == 1'))
    # PBB-ECS-1 - Wall Atoms Color:
    pipeline.modifiers.append(AssignColorModifier(color = [0.3333333333333333, 0.0, 0.0]))
    # PBB-ECS-1 - Clear selection:
    pipeline.modifiers.append(ClearSelectionModifier())
    # PBB-ECS-1 - Select Top Beads:
    pipeline.modifiers.append(ExpressionSelectionModifier(expression = 'ParticleType == 3'))
    # PBB-ECS-1 - Top Bead Color:
    pipeline.modifiers.append(AssignColorModifier(color = [1.0, 1.0, 0.0]))
    # PBB-ECS-1 - Clear selection:
    pipeline.modifiers.append(ClearSelectionModifier())
    # PBB-ECS-1 - Select Bot Beads:
    pipeline.modifiers.append(ExpressionSelectionModifier(expression = 'ParticleType == 2'))
    # PBB-ECS-1 - Bottom Bead Color:
    pipeline.modifiers.append(AssignColorModifier(color = [0.6666666666666666, 1.0, 0.4980392156862745]))
    # PBB-ECS-1 - Clear selection:
    pipeline.modifiers.append(ClearSelectionModifier())
    # PBB-ECS-1 - Select Solvent Beads:
    pipeline.modifiers.append(ExpressionSelectionModifier(expression = 'ParticleType == 4'))
    # PBB-ECS-1 - Solvent Bead Color:
    pipeline.modifiers.append(AssignColorModifier(color = [0.6666666666666666, 1.0, 1.0]))
    # PBB-ECS-1 - Clear selection:
    pipeline.modifiers.append(ClearSelectionModifier())
    # PBB-ECS-1 - Wall Atoms (remove):
    pipeline.modifiers.append(ExpressionSelectionModifier(
        expression = 'MoleculeIdentifier == 0', 
        enabled = False))
    # PBB-ECS-1 - Top Beads (remove):
    pipeline.modifiers.append(ExpressionSelectionModifier(
        expression = '(ParticleIdentifier <= N/2 || Selection)', 
        enabled = False))
    # PBB-ECS-1 - Molecule:
    pipeline.modifiers.append(ExpressionSelectionModifier(
        expression = '( MoleculeIdentifier!= 1 || Selection)', 
        enabled = False))
    # PBB-ECS-1 - Delete selected:
    pipeline.modifiers.append(DeleteSelectedModifier())
    # PBB-ECS-1 - Coordination analysis:
    pipeline.modifiers.append(CoordinationAnalysisModifier(
        cutoff = 4.6, 
        number_of_bins = 249))
    if mol > 0:
        # PBB-ECS-1 - Expression selection:
        pipeline.modifiers.append(ExpressionSelectionModifier(expression = 'MoleculeIdentifier == {0}'.format(mol)))
        # PBB-ECS-1 - Assign color:
        pipeline.modifiers.append(AssignColorModifier(color = [0.0, 0.0, 1.0]))
        # PBB-ECS-1 - Clear selection:
        pipeline.modifiers.append(ClearSelectionModifier(enabled = False))
    
        # PBB-ECS-1 - Expression selection:
        pipeline.modifiers.append(ExpressionSelectionModifier(expression = 'MoleculeIdentifier != {0} && MoleculeIdentifier != 0'.format(mol)))
    else:
        pipeline.modifiers.append(ExpressionSelectionModifier(expression = 'MoleculeIdentifier != 0'.format(mol)))

    
    # PBB-ECS-1 - Compute property:
    pipeline.modifiers.append(ComputePropertyModifier(
        expressions = ('{0}'.format(transp),), 
        output_property = 'Transparency', 
        only_selected = True))
    # PBB-ECS-1 - Clear selection:
    pipeline.modifiers.append(ClearSelectionModifier())
    
    # Expression selection:
    pipeline.modifiers.append(ExpressionSelectionModifier(expression = 'fmod(MoleculeIdentifier,2) == 0 && MoleculeIdentifier != 0'))
    interactions = 0
    beads = 1
    if cal_ints:
        # Compute property:
        pipeline.modifiers.append(ComputePropertyModifier(
            output_property = 'Interactions', 
            only_selected = True, 
            cutoff_radius = cutoff, 
            neighbor_expressions = ('fmod(MoleculeIdentifier,2) == 1',)))
        def get_interactions(frame, data):
            interactions = data.particles["Interactions"]
            data.attributes["Total Interactions"] = np.sum(interactions)
        pipeline.modifiers.append(get_interactions)
        data = pipeline.compute()
        interactions = data.attributes["Total Interactions"]
        beads = data.attributes["ExpressionSelection.count.2"] * 2

        del data
    
    return interactions, beads

In [7]:
flex_layout = widgets.Layout(display='flex',flex_flow='row wrap',align_items='stretch',border='2px solid gray')
full_layout = widgets.Layout(border='2px solid gray',width='100%')
half_layout = widgets.Layout(border='2px solid gray',width='50%')

# Shear Snapshots

In [8]:
TR_info_f = 'data/T_info'
with open(TR_info_f,'rb') as f:
        unpickler = pickle.Unpickler(f)
        TR_info = unpickler.load()
TR_Names = [TR_info[i][0].split()[0] for i in range(len(TR_info))]
TR_Ndirs = TR_info[0][1]
TR_Mdirs = TR_info[0][2]
TR_Xdirs = TR_info[0][3]
TR_PDdirs = TR_info[0][4]
TR_Vdirs = TR_info[0][5]

In [ ]:
if os.path.isdir('animations') == False:
    os.mkdir('animations')
    
#Thermo = widgets.Dropdown(options = ['Wall','Langevin','DPDT'], description='Thermostat')
Arch = widgets.Dropdown(options = TR_Names, description='Architecture')
N=widgets.SelectionSlider(options = TR_Ndirs, description='Chain Length', continuous_update=False)
M=widgets.SelectionSlider(options= TR_Mdirs, description='Grafting density', continuous_update=False)
Kb=widgets.SelectionSlider(options = TR_Xdirs, description='Chain stiffness', continuous_update=False)
PD=widgets.SelectionSlider(options = TR_PDdirs, description='Wall Seperation', continuous_update=True)
V=widgets.SelectionSlider(options = TR_Vdirs, description='Shear Velocity', continuous_update=True)
mol = widgets.IntSlider(min=-1,max= 89, value = 75, step = 2, description='Molecule', continuous_update=True)
transp = widgets.FloatSlider(min=0.00,max= 1.00, value = 0.00, step = 0.05, description='Transparency', continuous_update=False)
coords = widgets.Checkbox(value=False,indent=False,layout=widgets.Layout(width='27%',height='25%',margin='2px 0 0 20px'),description='Coords')
View = widgets.Dropdown(options = ['Front','Perspective'], description='View')
Animation = widgets.Button(description='OutputAnimation')
Image = widgets.Button(description='OutputImage')
CoF = widgets.FloatText(value=0, description='CoF',layout=widgets.Layout(width='45%'))
Rgx = widgets.FloatText(value=0, description='Rgx',layout=widgets.Layout(width='45%'))
T = widgets.FloatText(value=0, description='Temp',layout=widgets.Layout(width='45%'))
CoF_e = widgets.FloatText(value=0, description='+/-',layout=widgets.Layout(width='40%'))
Rgx_e = widgets.FloatText(value=0, description='+/-',layout=widgets.Layout(width='40%'))
T_e = widgets.FloatText(value=0, description='+/-',layout=widgets.Layout(width='40%'))
cal_ints = widgets.Checkbox(value=False,indent=False,layout=widgets.Layout(width='27%',height='25%',margin='2px 0 0 20px'))
cutoff = widgets.FloatText(value=1.5,indent=False, description='cutoff',layout=widgets.Layout(width='45%'))
intsperbead = widgets.FloatText(value=0, description='ints/bead',layout=widgets.Layout(width='40%'))

topo_box = widgets.Box(children=[Arch,N,M,Kb,PD,V],layout=flex_layout)
CoF_box = widgets.HBox(children=[CoF,CoF_e])
Rgx_box = widgets.HBox(children=[Rgx,Rgx_e])
T_box = widgets.HBox(children=[T,T_e])
I_box = widgets.HBox(children=[cal_ints,cutoff,intsperbead])
thermo_box = widgets.VBox(children=[CoF_box,Rgx_box,T_box,I_box],layout=half_layout)
vis_box = widgets.VBox(children=[transp,mol,coords,View,Animation,Image],layout=half_layout)
ui = widgets.Box(children = [topo_box,thermo_box,vis_box], layout = flex_layout)

def output_animation(b):
    pipeline = import_file(brush_path, multiple_frames = True)
    interactions, beads = my_mod(pipeline,mol2,cutoff2,cal_ints2,transp)
    frames = pipeline.source.num_frames
    if view2 == 'Front':
        vp = Viewport(type = Viewport.Type.Front)
    else:
        vp = Viewport(type = Viewport.Type.Perspective)
        vp.camera_dir = (-0.8, 1.4, -0.4)
    if coords:
        tripod = CoordinateTripodOverlay()
        tripod.size = 0.07
        tripod.style = CoordinateTripodOverlay.Style.Solid
        vp.overlays.append(tripod)
    vp.zoom_all()
    vp.render_anim(filename=os.path.join('animations',animation_name), size=(1200, 1200), renderer=renderer, fps=5)
    pipeline.remove_from_scene()
    IPython.display.Video(os.path.join('animations',animation_name),embed=True)
    
def output_image(b):
    if os.path.isdir('images') == False:
        os.mkdir('images')
    copyfile('image.png',os.path.join('images',image_name))

def update_Rvis(Arch,N,M,Kb,PD,V,transp,mol,View,cutoff,cal_ints,coords):
    global brush_path,animation_name,image_name,renderer,mol2,cutoff2, cal_ints2, view2
    # Data import:
    sims_path = os.path.join('..',Arch,'Sims')
    brush_path = os.path.join(sims_path,N,M,Kb,PD,'PBB-ECS',V,'shear.dump')
    log_path = os.path.join(sims_path,N,M,Kb,PD,'PBB-ECS',V,'shear.log')
#     print(log_path)
    ####################
    if os.path.isfile(log_path):
        log = pb.Log(log_path)
        data = log.get_data((1,25,30))
        
        T.value = round(data[0,0],4)
        T_e.value = round(data[0,1],4)
        Rgx.value = round(data[1,0],4)
        Rgx_e.value = round(data[1,1],4)
        CoF.value = data[2,0]
        CoF_e.value = data[2,1]
#         print(data)
#         CoF.value = round(data[2,0],4)
#         CoF_e.value = round(data[2,1],4)
    ####################
    pipeline = import_file(brush_path, multiple_frames = True)
    # Apply my modifications
    mol2 = mol
    cutoff2 = cutoff
    cal_ints2 = cal_ints
    view2 = View
    interactions, beads = my_mod(pipeline,mol,cutoff,cal_ints,transp) 
    intspbead =  interactions/beads
    intsperbead.value = intspbead    
    frames = pipeline.source.num_frames
    # Viewport setup:
    if View == 'Front':
        vp = Viewport(type = Viewport.Type.Front)
    else:
        vp = Viewport(type = Viewport.Type.Perspective)
        vp.camera_dir = (-0.8, 1.4, -0.4)
    animation_name = '{0}-{1}-{2}-{3}-{4}-{5}.avi'.format(Arch,N,M,Kb,PD,V)
    image_name = '{0}-{1}-{2}-{3}-{4}-{5}.png'.format(Arch,N,M,Kb,PD,V)
    if coords:
        tripod = CoordinateTripodOverlay()
        tripod.size = 0.07
        tripod.style = CoordinateTripodOverlay.Style.Solid
        vp.overlays.append(tripod)
    vp.zoom_all()
    # Renderer setup:
#     renderer=TachyonRenderer(ambient_occlusion=False, shadows=False)
    renderer = OpenGLRenderer(antialiasing_level = 3)
    vp.render_image(filename='image.png', size=(1410, 1200), renderer=renderer, frame=frames-1)
    pipeline.remove_from_scene()
    plt.figure(figsize=(16, 20), dpi=200)
    ax = plt.gca()
    ax.axes.xaxis.set_ticks([])
    ax.axes.yaxis.set_ticks([])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    img = mpimg.imread('image.png')
#     cropped_img = crop_image(img)
    imgplot = plt.imshow(img)

out =  widgets.interactive_output(update_Rvis,{'Arch': Arch,'N':N,'M':M,'Kb': Kb,'PD':PD,'V':V,'transp':transp,'mol': mol,'coords':coords,'View': View,'cutoff': cutoff,'cal_ints': cal_ints})
Animation.on_click(output_animation)
Image.on_click(output_image)
display(out,ui)
# Animation.on_click(test)



Output()

Box(children=(Box(children=(Dropdown(description='Architecture', options=('Cyclic', 'Linear', 'Loop', 'BB'), v…

In [10]:
HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')